# Pose Landmarks with MediaPipe — From Local Videos & Folders Using Python

This notebook is both a **guided lesson** and a **working pipeline** for detecting human pose landmarks from **local video files** or **entire folders** of videos using **MediaPipe Tasks**.

## Goal

1. Set up a clean Python 3.12 environment and verify required packages.
2. Understand each step and terminologies.
3. Download and select a Pose Landmarker model (**lite / full / heavy**) and understand accuracy–speed trade-offs.
4. Read videos with OpenCV and run inference in **`RunningMode.VIDEO`** with correct **timestamps**.
5. Export results as tidy CSVs for analysis: **2D image-normalized** and **3D world** landmarks.
6. Create an **annotated MP4** showing the skeleton overlay.
7. Build intuition for **visibility**, **image vs. world coordinates**, and simple feature engineering (e.g., joint angles).

> **Built for learning:** Along the way you’ll see short callouts explaining *why* each step exists (e.g., timestamps in VIDEO mode), how coordinate spaces differ, and how to tune speed vs. accuracy.

## After completing this guide, you will be able to

* Load one video—or loop through an entire folder—and extract the coordinates of the landmark bodypoints frame-by-frame.
* Save two analysis-ready CSVs per video: one for **2D normalized** landmarks and one for **3D world** coordinates.
* Produce an **annotated MP4** with landmarks and connections overlaid.
* Explain and adjust **`RunningMode.VIDEO`**, **per-frame timestamps**, **visibility filtering**, **image vs. world coordinates**, and model variants (**lite/full/heavy**).

> **Prerequisites**
>
> * Python **3.12** virtual environment selected as the active Jupyter kernel. In case yo8u need help, please refer to the "LS100_Guide 3_Introduction to Pose Estimation Using MediaPipe.pdf" guide.
> * Installed packages: `mediapipe opencv-python pandas numpy tqdm matplotlib seaborn`
> * One or more local video files (e.g., `.mp4`) to test.

> **Ethics & consent**
>
> * If processing videos of people, obtain consent and store data securely. Avoid uploading sensitive content to third-party services.

### References for learners

* MediaPipe Pose Landmarker (Python guide): [https://ai.google.dev/edge/mediapipe/solutions/vision/pose_landmarker/python](https://ai.google.dev/edge/mediapipe/solutions/vision/pose_landmarker/python)
* Pose Landmarker API: [https://ai.google.dev/edge/api/mediapipe/python/mp/tasks/vision/PoseLandmarker](https://ai.google.dev/edge/api/mediapipe/python/mp/tasks/vision/PoseLandmarker)
* Model card (BlazePose GHUM 3D; lite/full/heavy):
  [https://storage.googleapis.com/mediapipe-assets/Model%20Card%20BlazePose%20GHUM%203D.pdf](https://storage.googleapis.com/mediapipe-assets/Model%20Card%20BlazePose%20GHUM%203D.pdf)

---


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 0. Environment Setup and Verification (LS100 Standard)

Before running any code, make sure you’re using the **LS100_PoseEstimation_MP** kernel that was created in your Python 3.12 virtual environment.
This section verifies your environment and installs all required packages.

---

### **What you should already have**

✅ Python 3.12 installed

✅ Virtual environment activated (`(MediaPipeEnv)` should appear in your terminal)

✅ Kernel registered as **LS100_PoseEstimation_MP**

If you haven’t completed those steps, revisit the **LS100_Guide 3_Introduction to Pose Estimation Using MediaPipe.pdf** document.

---

### **Required packages**

This notebook uses the following libraries:

* `mediapipe` – pose landmark model and API
* `opencv-python` – video I/O (input/output) and frame conversion
* `pandas` & `numpy` – data handling and analysis
* `tqdm` – progress bars for video processing
* `matplotlib` & `seaborn` – visualization and data inspection

Run the next cell to ensure these are installed and to confirm the environment details.

---

### **Learning focus**

* Why virtual environments prevent version conflicts
* Why we require **Python 3.12** (MediaPipe Tasks currently supports Python 3.9–3.12 only)
* How each library fits into the MediaPipe Pose pipeline
---



## 0. Environment setup

> If running locally (VS Code/Jupyter), run the following cell once; it might take about a minute to run.


In [ ]:
# ============================================
# 0. Environment Setup and Package Verification
# ============================================

import sys
import importlib
import subprocess

# ---- 1. Check Python version ----
py_version = sys.version_info
print(f"🧠 Python version: {py_version.major}.{py_version.minor}.{py_version.micro}")
if py_version < (3, 9) or py_version >= (3, 13):
    print("⚠️ MediaPipe Tasks officially supports Python 3.9–3.12.")
    print("⚠️ Please switch to Python 3.12 for this notebook (as used in LS100).")

# ---- 2. Define required packages ----
required_packages = [
    "mediapipe",
    "opencv-python",
    "pandas",
    "numpy",
    "tqdm",
    "matplotlib",
    "seaborn",
]

# ---- 3. Function to check and install ----
def install_if_missing(pkg):
    """
    Try importing the package; if not found, install it quietly.
    """
    try:
        importlib.import_module(pkg.split("==")[0])
        print(f"✅ {pkg} already installed")
    except ImportError:
        print(f"⬇️ Installing {pkg} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

# ---- 4. Verify each dependency ----
for package in required_packages:
    install_if_missing(package)

# ---- 5. Print package versions for reproducibility ----
import mediapipe as mp
import cv2, pandas as pd, numpy as np, tqdm, matplotlib, seaborn

print("\n📦 Package versions:")
print(f"mediapipe      : {mp.__version__}")
print(f"opencv-python  : {cv2.__version__}")
print(f"pandas         : {pd.__version__}")
print(f"numpy          : {np.__version__}")
print(f"matplotlib     : {matplotlib.__version__}")
print(f"seaborn        : {seaborn.__version__}")

print("\n✅ Environment is ready to proceed!")


🧠 Python version: 3.12.12
⬇️ Installing mediapipe ...
⬇️ Installing opencv-python ...
✅ pandas already installed
✅ numpy already installed
✅ tqdm already installed
✅ matplotlib already installed
✅ seaborn already installed


/usr/local/lib/python3.12/dist-packages/jaxlib/plugin_support.py:71: RuntimeWarning: JAX plugin jax_cuda12_plugin version 0.7.2 is installed, but it is not compatible with the installed jaxlib version 0.7.1, so it will not be used.
  warnings.warn(



📦 Package versions:
mediapipe      : 0.10.21
opencv-python  : 4.11.0
pandas         : 2.2.2
numpy          : 2.0.2
matplotlib     : 3.10.0
seaborn        : 0.13.2

✅ Environment is ready to proceed!



## 1. Imports & version checks


---
# 1. Imports and Version Verification

Now that your environment is ready, let’s import the main libraries used throughout this notebook.

This step helps confirm that:

* The correct packages are installed inside your LS100 virtual environment
* MediaPipe loads successfully (and we can access its **Tasks API**)
* OpenCV, NumPy, and Pandas are working properly

If an import fails, it usually means you’re running the notebook in a different kernel (not the one you registered).
You can fix that by selecting **Kernel → Change Kernel → LS100_PoseEstimation_MP** (or the name you chose).

---

In [ ]:
# ======================================
# 1. Import Libraries and Verify Versions (fixed for MediaPipe >=0.10)
# ======================================

import os, cv2, numpy as np, pandas as pd, matplotlib, seaborn as sns
from tqdm import tqdm

import mediapipe as mp
from mediapipe.tasks import python as mp_python
from mediapipe.tasks.python import vision as mp_vision

print("✅ MediaPipe Tasks API imported successfully!\n")
print(f"mediapipe version : {mp.__version__}")
print(f"opencv version    : {cv2.__version__}")
print(f"pandas version    : {pd.__version__}")
print(f"numpy version     : {np.__version__}")

# Optional: check GPU availability
backend = "GPU" if cv2.cuda.getCudaEnabledDeviceCount() > 0 else "CPU"
print(f"⚙️ Running on {backend}")

# ---- Smoke test: confirm Tasks API symbols exist ----
BaseOptions = mp_python.BaseOptions
PoseLandmarker = mp_vision.PoseLandmarker
PoseLandmarkerOptions = mp_vision.PoseLandmarkerOptions
RunningMode = mp_vision.RunningMode

print("\n MediaPipe Tasks API is available:")
print(f"- BaseOptions           : {BaseOptions is not None}")
print(f"- PoseLandmarker        : {PoseLandmarker is not None}")
print(f"- PoseLandmarkerOptions : {PoseLandmarkerOptions is not None}")
print(f"- RunningMode           : {RunningMode is not None}")

✅ MediaPipe Tasks API imported successfully!

mediapipe version : 0.10.21
opencv version    : 4.11.0
pandas version    : 2.2.2
numpy version     : 2.0.2
⚙️ Running on CPU

 MediaPipe Tasks API is available:
- BaseOptions           : True
- PoseLandmarker        : True
- PoseLandmarkerOptions : True
- RunningMode           : True


---

### 💡 **Notes**

* **Why this matters:** ensures that the environment is truly isolated and reproducible.
* **Discussion prompt:** Can you tell *why* we check MediaPipe imports *before* running the pipeline? (to confirm the **Tasks** API is available and working).
* **TASK:** Print `mp.__file__` to confirm MediaPipe’s path. This helps you understand where packages live inside the venv.

---


## 2. How Pose Landmarker works (what to teach)

- **Running modes:** `IMAGE`, `VIDEO`, `LIVE_STREAM`. For offline videos we use **`VIDEO`** and must pass a **timestamp (ms)** for each frame; the task uses **tracking** to avoid re-running the full model on every frame (reduces latency at the same accuracy settings).  
- **Outputs:**  
  - **2D normalized landmarks** in image coordinates (*x,y in [0,1] relative to width/height; z is a depth-like value; visibility in [0,1]*).  
  - **3D world landmarks** (meters, origin near hip center; handy for biomechanical features).  
- **Variants:** **lite / full / heavy**. Heavier models = more accurate, slower (see model card).  
- **Accuracy vs speed knobs:** `num_poses` (usually 1 for single-person), `min_pose_detection_confidence`, `min_pose_presence_confidence`, `min_tracking_confidence`, and **frame stride** (e.g., analyze every 2nd/3rd frame).

> We’ll expose all of these transparently in helper functions below.



## 3. Download a Pose Landmarker model (`.task` bundle)

Choose one of: `"lite"`, `"full"` (default), `"heavy"`.  
URLs follow Google’s published pattern; we try `latest/…` first and then fall back to version `1/…`.

> You only need to download once; it will be cached under `models/`.


In [ ]:
# ================================
# 2. Model Selection & Download
# ================================
import os
import pathlib
import urllib.request
import urllib.error

import mediapipe as mp
from mediapipe.tasks import python as mp_python
from mediapipe.tasks.python import vision as mp_vision

# ---- Where to save models ----
MODELS_DIR = pathlib.Path("models")
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# ---- Official model URLs (latest, with fallback to v1) ----
MODEL_URLS = {
    "lite": [
        "https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_lite/float16/latest/pose_landmarker_lite.task",
        "https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_lite/float16/1/pose_landmarker_lite.task",
    ],
    "full": [
        "https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_full/float16/latest/pose_landmarker_full.task",
        "https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_full/float16/1/pose_landmarker_full.task",
    ],
    "heavy": [
        "https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/latest/pose_landmarker_heavy.task",
        "https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task",
    ],
}

def download_pose_model(variant: str = "full") -> str:
    """
    Download the selected model variant (.task) to MODELS_DIR.
    Returns the local file path.
    """
    variant = variant.lower().strip()
    assert variant in MODEL_URLS, f"Unknown variant '{variant}'. Choose: lite, full, heavy."

    out_path = MODELS_DIR / f"pose_landmarker_{variant}.task"
    if out_path.exists() and out_path.stat().st_size > 50_000:
        print(f"✔ Model already present: {out_path}")
        return str(out_path)

    last_err = None
    for url in MODEL_URLS[variant]:
        try:
            print(f"Downloading {variant} model from:\n  {url}")
            with urllib.request.urlopen(url, timeout=60) as r, open(out_path, "wb") as f:
                f.write(r.read())
            if out_path.stat().st_size <= 50_000:
                raise RuntimeError("Downloaded file seems too small; trying fallback...")
            print(f"✔ Saved to {out_path} ({out_path.stat().st_size/1e6:.2f} MB)")
            return str(out_path)
        except Exception as e:
            print(f"… failed: {e}")
            last_err = e
    raise RuntimeError(f"Could not download model for variant '{variant}'. Last error: {last_err}")

# ---- Choose your default model here ----
MODEL_VARIANT = "full"   # options: "lite", "full", "heavy"
MODEL_PATH = download_pose_model(MODEL_VARIANT)

# ---- Verify we can initialize the Pose Landmarker (VIDEO mode) ----
BaseOptions = mp_python.BaseOptions
PoseLandmarker = mp_vision.PoseLandmarker
PoseLandmarkerOptions = mp_vision.PoseLandmarkerOptions
RunningMode = mp_vision.RunningMode

options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=MODEL_PATH),
    running_mode=RunningMode.VIDEO,
    num_poses=1,
    min_pose_detection_confidence=0.5,
    min_pose_presence_confidence=0.5,
    min_tracking_confidence=0.5,
    output_segmentation_masks=False,
)

try:
    with PoseLandmarker.create_from_options(options) as landmarker:
        print("✅ PoseLandmarker initialized successfully (VIDEO mode).")
        print(f"   Model: {MODEL_VARIANT} → {MODEL_PATH}")
except Exception as e:
    print("❌ Failed to initialize PoseLandmarker. Check the model file and MediaPipe version.")
    raise


  https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_full/float16/latest/pose_landmarker_full.task
✔ Saved to models/pose_landmarker_full.task (9.40 MB)
✅ PoseLandmarker initialized successfully (VIDEO mode).
   Model: full → models/pose_landmarker_full.task


## 4. VIDEO mode: timestamps & inference loop

For offline videos, we must use RunningMode.VIDEO and pass a monotonic timestamp (ms) for each frame:

* We read frames with OpenCV, compute timestamp_ms = int((frame_idx / fps) * 1000), and call
landmarker.detect_for_video(mp_image, timestamp_ms).

* The Task returns normalized 2D landmarks (x, y ∈ [0,1], z depth-like, plus visibility) and world 3D landmarks (x_m, y_m, z_m in meters).

* We’ll save tidy CSV files for 2D and 3D landmarks.

* We’ll also write an annotated MP4 by drawing a simple skeleton over each frame.

#### Parameters you can tune

* `MODEL_VARIANT` (lite/full/heavy), `num_poses` (usually 1), `frame_stride` (skip frames for speed),

* `min_pose_detection_confidence`, `min_pose_presence_confidence`, `min_tracking_confidence`.

# 5 Choose Your Parameters

Before running extraction, set the **tunable parameters** in the next cell.
These variables control the accuracy, speed, and outputs of your pipeline.

---

### **Key Parameters**

* **`MODEL_VARIANT`** — choose one of:

  * `lite` → fastest but lowest accuracy
  * `full` → balanced and recommended default
  * `heavy` → most accurate but slower

* **`frame_stride`** — process every *k*-th frame.

  * `1` = analyze every frame (slow but detailed)
  * `2` = every other frame (twice as fast)
  * `3+` = even faster but less temporal precision

* **`num_poses`** — number of people to detect per frame.

  * Use `1` for single-person videos (default for LS100).

* **Confidence thresholds**

  * `min_pose_detection_confidence` → how certain a pose must be detected
  * `min_pose_presence_confidence` → confidence threshold for the person’s presence
  * `min_tracking_confidence` → how reliably MediaPipe should track the same pose across frames

* **`make_annotated_video`** — if `True`, saves a new video (`.mp4`) with skeleton overlays.

* **`out_dir`** — the directory where all **CSVs** and (optional) **annotated MP4s** will be saved.

---

> 🧠 **Tip:** If you change `MODEL_VARIANT`, the model will automatically download the correct `.task` file again.
> Choose the settings thoughtfully based on your hardware and analysis needs—students with slower laptops should prefer `lite` and/or increase `frame_stride`.

---

In [ ]:
# =========================================
# 2.1 Parameters — YOU control the pipeline
# =========================================

# --- Model choice ---
MODEL_VARIANT = "lite"          # options: "lite", "full", "heavy"

# --- Inference behavior ---
frame_stride = 2                 # 1=every frame; 2=every other; 3=every third; etc.
num_poses = 1                    # usually 1 for single-person videos
min_pose_detection_confidence = 0.5
min_pose_presence_confidence  = 0.5
min_tracking_confidence       = 0.5

# --- Outputs ---
make_annotated_video = True      # set False to skip saving annotated MP4
out_dir = "outputs"              # where CSVs / MP4 will be written

# --- Sync the model file if variant changed ---
# Assumes the download_pose_model() function from earlier cells is defined.
def ensure_model_variant(variant: str) -> str:
    v = variant.strip().lower()
    if v not in ("lite", "full", "heavy"):
        raise ValueError("MODEL_VARIANT must be one of: 'lite', 'full', 'heavy'.")
    return download_pose_model(v)

MODEL_PATH = "models/pose_landmarker_full.task"
print(f"✔ Using model: {MODEL_VARIANT} → {MODEL_PATH}")
print(f"   frame_stride={frame_stride}, num_poses={num_poses}, annotated={make_annotated_video}")
print(f"   confidences: detection={min_pose_detection_confidence}, presence={min_pose_presence_confidence}, tracking={min_tracking_confidence}")
print(f"   output dir: {out_dir}")


✔ Using model: lite → models/pose_landmarker_full.task
   frame_stride=2, num_poses=1, annotated=True
   confidences: detection=0.5, presence=0.5, tracking=0.5
   output dir: outputs


In [ ]:
# =========================================================
# 3.1 Ensure CSVs include landmark_name & add peek helpers
# =========================================================

# If you already defined POSE_LANDMARK_NAMES & landmark_index_to_name earlier, we reuse them.
# If not, define them quickly here:
try:
    landmark_index_to_name
except NameError:
    POSE_LANDMARK_NAMES = [
        "nose","left_eye_inner","left_eye","left_eye_outer",
        "right_eye_inner","right_eye","right_eye_outer",
        "left_ear","right_ear","mouth_left","mouth_right",
        "left_shoulder","right_shoulder","left_elbow","right_elbow",
        "left_wrist","right_wrist","left_pinky","right_pinky",
        "left_index","right_index","left_thumb","right_thumb",
        "left_hip","right_hip","left_knee","right_knee",
        "left_ankle","right_ankle","left_heel","right_heel",
        "left_foot_index","right_foot_index",
    ]
    landmark_index_to_name = {i: n for i, n in enumerate(POSE_LANDMARK_NAMES)}

# --- Re-define extract_pose_from_video to include `landmark_name` columns ---
def extract_pose_from_video(
    video_path: str,
    model_path: str,
    out_dir: str = "outputs",
    make_annotated_video: bool = False,
    frame_stride: int = 1,
    num_poses: int = 1,
    min_pose_detection_confidence: float = 0.5,
    min_pose_presence_confidence: float = 0.5,
    min_tracking_confidence: float = 0.5,
    output_segmentation_masks: bool = False,
):
    import os, pathlib
    import cv2, numpy as np, pandas as pd
    import mediapipe as mp
    from mediapipe.tasks import python as mp_python
    from mediapipe.tasks.python import vision as mp_vision

    os.makedirs(out_dir, exist_ok=True)
    stem = pathlib.Path(video_path).stem
    csv2d = os.path.join(out_dir, f"{stem}_pose2d.csv")
    csv3d = os.path.join(out_dir, f"{stem}_pose3d.csv")
    mp4_out = os.path.join(out_dir, f"{stem}_annotated.mp4")

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise FileNotFoundError(f"Cannot open video: {video_path}")

    fps    = cap.get(cv2.CAP_PROP_FPS) or 30.0
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    writer = None
    if make_annotated_video:
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        writer  = cv2.VideoWriter(mp4_out, fourcc, fps / max(1, frame_stride), (width, height))

    BaseOptions = mp_python.BaseOptions
    PoseLandmarker = mp_vision.PoseLandmarker
    PoseLandmarkerOptions = mp_vision.PoseLandmarkerOptions
    RunningMode = mp_vision.RunningMode

    options = PoseLandmarkerOptions(
        base_options=BaseOptions(model_asset_path=model_path),
        running_mode=RunningMode.VIDEO,
        num_poses=num_poses,
        min_pose_detection_confidence=min_pose_detection_confidence,
        min_pose_presence_confidence=min_pose_presence_confidence,
        min_tracking_confidence=min_tracking_confidence,
        output_segmentation_masks=output_segmentation_masks,
    )

    # Minimal overlay helper (same as earlier cell)
    def _mp_image_from_bgr(bgr):
        rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
        return mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb)

    def _draw_skeleton(bgr, norm_landmarks, visibility_thresh=0.5):
        h, w = bgr.shape[:2]
        pts = {}
        for i, lm in enumerate(norm_landmarks):
            vis = getattr(lm, "visibility", 1.0) or 0.0
            if vis >= visibility_thresh:
                x, y = int(lm.x * w), int(lm.y * h)
                pts[i] = (x, y)
                cv2.circle(bgr, (x, y), 2, (255, 255, 255), -1)
        for a, b in [
            (11,13),(13,15),(12,14),(14,16),(11,12),(23,24),(11,23),(12,24),
            (23,25),(25,27),(24,26),(26,28),(27,29),(29,31),(28,30),(30,32)
        ]:
            if a in pts and b in pts:
                cv2.line(bgr, pts[a], pts[b], (255, 255, 255), 2)

    rows2d, rows3d = [], []

    with PoseLandmarker.create_from_options(options) as landmarker:
        frame_idx = 0
        while True:
            ok, bgr = cap.read()
            if not ok:
                break
            if frame_stride > 1 and (frame_idx % frame_stride != 0):
                frame_idx += 1
                continue

            ts_ms = int((frame_idx / fps) * 1000.0)
            mp_image = _mp_image_from_bgr(bgr)
            result = landmarker.detect_for_video(mp_image, ts_ms)

            for pose_id, nlands in enumerate(result.pose_landmarks):
                # 2D
                for li, lm in enumerate(nlands):
                    rows2d.append({
                        "video": os.path.basename(video_path),
                        "frame": frame_idx,
                        "time_ms": ts_ms,
                        "landmark_index": li,
                        "landmark_name": landmark_index_to_name.get(li, str(li)),
                        "x": lm.x,
                        "y": lm.y,
                        "z": lm.z,
                        "visibility": getattr(lm, "visibility", np.nan),
                    })
                # 3D
                if len(result.pose_world_landmarks) > pose_id:
                    wlands = result.pose_world_landmarks[pose_id]
                    for li, lm in enumerate(wlands):
                        rows3d.append({
                            "video": os.path.basename(video_path),
                            "frame": frame_idx,
                            "time_ms": ts_ms,
                            "landmark_index": li,
                            "landmark_name": landmark_index_to_name.get(li, str(li)),
                            "x_m": lm.x,
                            "y_m": lm.y,
                            "z_m": lm.z,
                            "visibility": getattr(lm, "visibility", np.nan),
                        })

                if writer is not None and len(nlands) > 0:
                    bgr_draw = bgr.copy()
                    _draw_skeleton(bgr_draw, nlands, visibility_thresh=0.5)
                    writer.write(bgr_draw)

            frame_idx += 1

    cap.release()
    if writer is not None:
        writer.release()

    pd.DataFrame(rows2d).to_csv(csv2d, index=False)
    if rows3d:
        pd.DataFrame(rows3d).to_csv(csv3d, index=False)
    else:
        csv3d = None

    return {"csv2d": csv2d, "csv3d": csv3d, "annotated_mp4": (mp4_out if make_annotated_video else None)}

# --- Quick peek helper ---
def peek_csv(path, n=5):
    import pandas as pd
    df = pd.read_csv(path)
    print(f"{path} → shape={df.shape}")
    display(df.head(n))
    return df

print("✅ Extraction function updated to include landmark_name in both CSVs.")


✅ Extraction function updated to include landmark_name in both CSVs.


In [11]:
# --- Quick usage example (edit the path) ---
# Example call using the “2.1 Parameters” variables the student set:
outputs = extract_pose_from_video(
    video_path="/content/drive/MyDrive/Harvard/LS100/videos/Frame_Reduced/clips/Aryeh_1_clip1.MP4", # Corrected to a specific video file
    model_path=MODEL_PATH,                # from the model download step
    out_dir=out_dir,
    make_annotated_video=False,
    frame_stride=frame_stride,
    num_poses=num_poses,
    min_pose_detection_confidence=min_pose_detection_confidence,
    min_pose_presence_confidence=min_pose_presence_confidence,
    min_tracking_confidence=min_tracking_confidence,
)
outputs

{'csv2d': 'outputs/Aryeh_1_clip1_pose2d.csv',
 'csv3d': 'outputs/Aryeh_1_clip1_pose3d.csv',
 'annotated_mp4': None}


> **Notes for students**
> - **2D normalized coordinates**: `x,y∈[0,1]` relative to image width/height (values can be outside the range if the estimated point is out-of-frame). `z` is depthlike (negative is closer).
> - **3D world coordinates**: `x,y,z` are in **meters** in a world coordinate space centered near the hips.  
> - **visibility**: confidence for each landmark’s presence in the frame.



## 6. Batch mode — process a directory of videos

Set `input_dir` and an optional `glob` pattern (e.g., `"*.mp4"`). Each video will
produce two CSVs (2D + 3D) and, if enabled, an annotated MP4.


In [12]:
# ============================================
# 4. Batch Processing: process a folder of videos
# ============================================
import os, glob, pathlib, pandas as pd
from typing import List, Dict, Optional
from tqdm import tqdm

# Common video extensions to search for (case-insensitive)
VIDEO_EXTS = (".mp4", ".mov", ".m4v", ".avi", ".mkv")

def list_videos(input_dir: str, recursive: bool = False) -> List[str]:
    """Return a sorted list of video file paths in the directory."""
    p = pathlib.Path(input_dir)
    if not p.exists() or not p.is_dir():
        raise NotADirectoryError(f"Not a directory: {input_dir}")
    if recursive:
        files = [str(f) for f in p.rglob("*") if f.suffix.lower() in VIDEO_EXTS]
    else:
        files = [str(f) for f in p.iterdir() if f.is_file() and f.suffix.lower() in VIDEO_EXTS]
    return sorted(files)

def batch_process_videos(
    input_dir: str,
    model_path: str,
    out_dir: str = "outputs",
    recursive: bool = False,
    # Extraction params (default to Section 2.1 variables if they exist)
    make_annotated_video: Optional[bool] = None,
    frame_stride: Optional[int] = None,
    num_poses: Optional[int] = None,
    min_pose_detection_confidence: Optional[float] = None,
    min_pose_presence_confidence: Optional[float] = None,
    min_tracking_confidence: Optional[float] = None,
) -> pd.DataFrame:
    """
    Process all videos in a directory and return a summary table
    with paths to the generated CSVs and (optional) annotated MP4s.
    """
    # Pull defaults from the Section 2.1 variables if not provided
    globals_fallbacks = {
        "make_annotated_video": True,
        "frame_stride": 1,
        "num_poses": 1,
        "min_pose_detection_confidence": 0.5,
        "min_pose_presence_confidence": 0.5,
        "min_tracking_confidence": 0.5,
    }
    g = globals()
    if make_annotated_video is None: make_annotated_video = g.get("make_annotated_video", globals_fallbacks["make_annotated_video"])
    if frame_stride is None: frame_stride = g.get("frame_stride", globals_fallbacks["frame_stride"])
    if num_poses is None: num_poses = g.get("num_poses", globals_fallbacks["num_poses"])
    if min_pose_detection_confidence is None: min_pose_detection_confidence = g.get("min_pose_detection_confidence", globals_fallbacks["min_pose_detection_confidence"])
    if min_pose_presence_confidence is None:  min_pose_presence_confidence  = g.get("min_pose_presence_confidence",  globals_fallbacks["min_pose_presence_confidence"])
    if min_tracking_confidence is None:       min_tracking_confidence       = g.get("min_tracking_confidence",       globals_fallbacks["min_tracking_confidence"])

    os.makedirs(out_dir, exist_ok=True)
    files = list_videos(input_dir, recursive=recursive)
    if not files:
        raise FileNotFoundError(f"No supported video files found in: {input_dir}")

    records: List[Dict[str, Optional[str]]] = []
    print(f"Found {len(files)} video(s) in {input_dir} (recursive={recursive}).\n")

    for fpath in tqdm(files, desc="Batch processing", unit="video"):
        try:
            outs = extract_pose_from_video(
                video_path=fpath,
                model_path=model_path,
                out_dir=out_dir,
                make_annotated_video=make_annotated_video,
                frame_stride=frame_stride,
                num_poses=num_poses,
                min_pose_detection_confidence=min_pose_detection_confidence,
                min_pose_presence_confidence=min_pose_presence_confidence,
                min_tracking_confidence=min_tracking_confidence,
            )
            records.append({
                "video": fpath,
                "csv2d": outs.get("csv2d"),
                "csv3d": outs.get("csv3d"),
                "annotated_mp4": outs.get("annotated_mp4"),
                "status": "ok",
                "error": "",
            })
        except Exception as e:
            records.append({
                "video": fpath,
                "csv2d": None,
                "csv3d": None,
                "annotated_mp4": None,
                "status": "error",
                "error": str(e),
            })

    df = pd.DataFrame.from_records(records)
    # Optional: save a manifest for later reference
    manifest_path = os.path.join(out_dir, "batch_manifest.csv")
    df.to_csv(manifest_path, index=False)
    print(f"\n✔ Batch complete. Manifest saved to: {manifest_path}")
    display(df)
    return df


In [13]:
# --- Example usage (edit the path to your folder) ---
df_summary = batch_process_videos(
     input_dir="/content/drive/MyDrive/Harvard/LS100/videos/Frame_Reduced/clips",
     model_path=MODEL_PATH,    # from model download step
     out_dir=out_dir,          # from Section 2.1
     recursive=False,          # set True to search subfolders
 )
df_summary.head()

Found 15 video(s) in /content/drive/MyDrive/Harvard/LS100/videos/Frame_Reduced/clips (recursive=False).



Batch processing: 100%|██████████| 15/15 [29:00<00:00, 116.04s/video]


✔ Batch complete. Manifest saved to: outputs/batch_manifest.csv


,video,csv2d,csv3d,annotated_mp4,status,error
0,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Aryeh_1_clip1_pose2d.csv,outputs/Aryeh_1_clip1_pose3d.csv,outputs/Aryeh_1_clip1_annotated.mp4,ok,
1,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Aryeh_1_clip2_pose2d.csv,outputs/Aryeh_1_clip2_pose3d.csv,outputs/Aryeh_1_clip2_annotated.mp4,ok,
2,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Aryeh_1_clip3_pose2d.csv,outputs/Aryeh_1_clip3_pose3d.csv,outputs/Aryeh_1_clip3_annotated.mp4,ok,
3,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Aryeh_2_clip4_pose2d.csv,outputs/Aryeh_2_clip4_pose3d.csv,outputs/Aryeh_2_clip4_annotated.mp4,ok,
4,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Aryeh_2_clip5_pose2d.csv,outputs/Aryeh_2_clip5_pose3d.csv,outputs/Aryeh_2_clip5_annotated.mp4,ok,
5,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Danny_1_clip1_pose2d.csv,outputs/Danny_1_clip1_pose3d.csv,outputs/Danny_1_clip1_annotated.mp4,ok,
6,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Danny_1_clip2_pose2d.csv,outputs/Danny_1_clip2_pose3d.csv,outputs/Danny_1_clip2_annotated.mp4,ok,
7,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Danny_1_clip3_pose2d.csv,outputs/Danny_1_clip3_pose3d.csv,outputs/Danny_1_clip3_annotated.mp4,ok,
8,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Danny_2_clip4_pose2d.csv,outputs/Danny_2_clip4_pose3d.csv,outputs/Danny_2_clip4_annotated.mp4,ok,
9,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Danny_2_clip5_pose2d.csv,outputs/Danny_2_clip5_pose3d.csv,outputs/Danny_2_clip5_annotated.mp4,ok,


,video,csv2d,csv3d,annotated_mp4,status,error
0,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Aryeh_1_clip1_pose2d.csv,outputs/Aryeh_1_clip1_pose3d.csv,outputs/Aryeh_1_clip1_annotated.mp4,ok,
1,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Aryeh_1_clip2_pose2d.csv,outputs/Aryeh_1_clip2_pose3d.csv,outputs/Aryeh_1_clip2_annotated.mp4,ok,
2,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Aryeh_1_clip3_pose2d.csv,outputs/Aryeh_1_clip3_pose3d.csv,outputs/Aryeh_1_clip3_annotated.mp4,ok,
3,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Aryeh_2_clip4_pose2d.csv,outputs/Aryeh_2_clip4_pose3d.csv,outputs/Aryeh_2_clip4_annotated.mp4,ok,
4,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Aryeh_2_clip5_pose2d.csv,outputs/Aryeh_2_clip5_pose3d.csv,outputs/Aryeh_2_clip5_annotated.mp4,ok,



## 7. Optional: compute simple joint angles

Once you have landmarks, you can compute feature engineering targets like **elbow** or **knee angles**. Below is a tiny utility to compute an angle between three named landmarks per frame.


In [14]:

def _angle_between(a, b, c):
    # a,b,c are 2D points (x,y) or 3D (x,y,z) — here we'll use 2D image coords
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosang = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-9)
    cosang = np.clip(cosang, -1.0, 1.0)
    return np.degrees(np.arccos(cosang))

def compute_joint_angle_csv(csv2d_path: str, joint=("left_shoulder","left_elbow","left_wrist")) -> pd.DataFrame:
    df = pd.read_csv(csv2d_path)
    # wide pivot: columns like x_left_shoulder, y_left_shoulder, etc.
    wide = df.pivot_table(index=["video","frame","time_ms"], columns="landmark_name", values=["x","y"])
    # helper to get a point
    def P(name):
        return np.c_[wide["x"][name].values, wide["y"][name].values]
    A,B,C = P(joint[0]), P(joint[1]), P(joint[2])
    angles = np.array([_angle_between(a,b,c) for a,b,c in zip(A,B,C)])
    out = pd.DataFrame({
        "video": wide.index.get_level_values("video"),
        "frame": wide.index.get_level_values("frame"),
        "time_ms": wide.index.get_level_values("time_ms"),
        f"angle_{'_'.join(joint)}": angles
    })
    return out

# Example (after extraction):
# angle_df = compute_joint_angle_csv("outputs/yourvideo_pose2d.csv", ("left_shoulder","left_elbow","left_wrist"))
# angle_df.head()



## 8. Notes & best practices

- **Timestamps matter:** in `VIDEO` mode you *must* pass `timestamp_ms` that increases with frames; we compute it from frame index and FPS.  
- **Tracking saves compute:** in `VIDEO`/`LIVE_STREAM` the task performs pose tracking so the full model isn’t re-run every frame (helps latency).  
- **Out-of-frame landmarks:** 2D normalized `x,y` can be outside `[0,1]` if a joint is off‑screen; use `visibility` to filter.  
- **Model choice:** start with **full**, switch to **lite** for underpowered laptops or large batches, use **heavy** when you need the highest accuracy and can afford the speed.  
- **Stride:** a cheap speedup is `frame_stride=2` (½ the frames) or higher.  
- **Ethics & consent:** if students process videos of people, teach consent, privacy, and secure storage.



## 9. Quick start (edit the path and run)

Uncomment one of the calls below and set your paths.


In [18]:

#--- SINGLE VIDEO ---
#outputs = extract_pose_from_video(
#    video_path="/Users/zachbuller/Desktop/LS100/Trial_Videos/Danny_1.MP4",
#    model_path=MODEL_PATH,
#    make_annotated_video=True,
#    frame_stride=2,   # increase to 2/3 for faster processing
# )
#print(outputs)

#--- DIRECTORY ---
df = batch_process_videos(
    input_dir="/content/drive/MyDrive/Harvard/LS100/videos/Frame_Reduced/clips",
    model_path=MODEL_PATH,
    out_dir="outputs",
    make_annotated_video=False,
    frame_stride=2
)

df



Found 15 video(s) in /content/drive/MyDrive/Harvard/LS100/videos/Frame_Reduced/clips (recursive=False).



Batch processing: 100%|██████████| 15/15 [21:51<00:00, 87.42s/video]


✔ Batch complete. Manifest saved to: outputs/batch_manifest.csv


,video,csv2d,csv3d,annotated_mp4,status,error
0,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Aryeh_1_clip1_pose2d.csv,outputs/Aryeh_1_clip1_pose3d.csv,None,ok,
1,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Aryeh_1_clip2_pose2d.csv,outputs/Aryeh_1_clip2_pose3d.csv,None,ok,
2,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Aryeh_1_clip3_pose2d.csv,outputs/Aryeh_1_clip3_pose3d.csv,None,ok,
3,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Aryeh_2_clip4_pose2d.csv,outputs/Aryeh_2_clip4_pose3d.csv,None,ok,
4,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Aryeh_2_clip5_pose2d.csv,outputs/Aryeh_2_clip5_pose3d.csv,None,ok,
5,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Danny_1_clip1_pose2d.csv,outputs/Danny_1_clip1_pose3d.csv,None,ok,
6,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Danny_1_clip2_pose2d.csv,outputs/Danny_1_clip2_pose3d.csv,None,ok,
7,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Danny_1_clip3_pose2d.csv,outputs/Danny_1_clip3_pose3d.csv,None,ok,
8,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Danny_2_clip4_pose2d.csv,outputs/Danny_2_clip4_pose3d.csv,None,ok,
9,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Danny_2_clip5_pose2d.csv,outputs/Danny_2_clip5_pose3d.csv,None,ok,


,video,csv2d,csv3d,annotated_mp4,status,error
0,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Aryeh_1_clip1_pose2d.csv,outputs/Aryeh_1_clip1_pose3d.csv,None,ok,
1,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Aryeh_1_clip2_pose2d.csv,outputs/Aryeh_1_clip2_pose3d.csv,None,ok,
2,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Aryeh_1_clip3_pose2d.csv,outputs/Aryeh_1_clip3_pose3d.csv,None,ok,
3,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Aryeh_2_clip4_pose2d.csv,outputs/Aryeh_2_clip4_pose3d.csv,None,ok,
4,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Aryeh_2_clip5_pose2d.csv,outputs/Aryeh_2_clip5_pose3d.csv,None,ok,
5,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Danny_1_clip1_pose2d.csv,outputs/Danny_1_clip1_pose3d.csv,None,ok,
6,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Danny_1_clip2_pose2d.csv,outputs/Danny_1_clip2_pose3d.csv,None,ok,
7,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Danny_1_clip3_pose2d.csv,outputs/Danny_1_clip3_pose3d.csv,None,ok,
8,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Danny_2_clip4_pose2d.csv,outputs/Danny_2_clip4_pose3d.csv,None,ok,
9,/content/drive/MyDrive/Harvard/LS100/videos/Fr...,outputs/Danny_2_clip5_pose2d.csv,outputs/Danny_2_clip5_pose3d.csv,None,ok,



---

### Troubleshooting
- If you see `NoneType` for results, ensure the **model path exists** and your video actually contains a person.  
- If you get slowdowns or memory pressure, try `frame_stride=2` or the `"lite"` model.  
- On some platforms OpenCV MP4 writing may need codecs; if a saved video is empty, try a different `fourcc` (e.g., `cv2.VideoWriter_fourcc(*"avc1")`) or install `opencv-python-headless` alternatives.

Happy exploring!


In [21]:
!mv /content/outputs "/content/drive/MyDrive/Harvard/LS100/videos/Frame_Reduced/clips/annotated_outputs"
